## Algorithme naïf

In [1]:
def match(string, pattern):
    matches = []
    for i in range(len(string)):
        for j in range(len(pattern)):
            if string[i+j] != pattern[j]:
                break
        else:  # for ... else: Cette branche ne s'exécute que
               #               si on n'est pas sortis du for
               #               par un break
            matches.append(i)
    return matches

On télécharge dans la variable `genome` le génome de l'E. coli

In [2]:
from requests import get
from gzip import decompress

req = get('http://defeo.lu/M1-AlgoProg/tds/ecoli.gz')
genome = decompress(req.content).decode()[71:]

In [3]:
%time m = match(genome, 'GATTACA')

CPU times: user 2.44 s, sys: 0 ns, total: 2.44 s
Wall time: 2.44 s


In [4]:
m

[23254,
 80864,
 155458,
 181444,
 262735,
 313586,
 316375,
 322262,
 357439,
 386155,
 392966,
 393371,
 479364,
 481239,
 498552,
 511721,
 516538,
 541301,
 541503,
 564697,
 567926,
 569904,
 602022,
 605460,
 634557,
 638760,
 703856,
 713982,
 732147,
 736486,
 736866,
 738133,
 740543,
 757751,
 780029,
 783601,
 805575,
 816856,
 831159,
 841739,
 860171,
 877581,
 894395,
 919777,
 923017,
 933119,
 953797,
 959900,
 998178,
 1001183,
 1098941,
 1099825,
 1103333,
 1123212,
 1134014,
 1134921,
 1139457,
 1151824,
 1200846,
 1206930,
 1210459,
 1211165,
 1223289,
 1235369,
 1292565,
 1301128,
 1322228,
 1325600,
 1336802,
 1340902,
 1374809,
 1378000,
 1394678,
 1397355,
 1405313,
 1408309,
 1414001,
 1471399,
 1480451,
 1488567,
 1503663,
 1512769,
 1516963,
 1531037,
 1531857,
 1532267,
 1550532,
 1580750,
 1633194,
 1642824,
 1649022,
 1680615,
 1787843,
 1790424,
 1854854,
 1862884,
 1915544,
 1932774,
 1936043,
 1983959,
 1987662,
 2016680,
 2033633,
 2035141,
 2051285,
 

## Rabin – Karp

In [5]:
def rabin_karp(string, pattern, mod):
    # Dictionnaire pour traduction: Alphabet -> Entiers
    #
    # utiliser un dictionnaire semble être ce qu'il y a de plus efficace en Python
    pi = { 'A': 0, 'G': 1, 'C': 2, 'T': 3 }

    # Eh, oui, on peut définir des fonctions dans des fonctions en Python!
    # Cette fonction ne peut être appellée que à l'intérieur de rabin_karp
    def string_to_int(s):
        res = 0
        for c in s:
            res *= 4
            # Les variables locales de la fonction "parent" (ici `pi` et `mod`)
            # sont accessibles depuis cette fonction imbriquée
            res = (res + pi[c]) % mod
        return res

    # Initialisation et test du préfixe
    matches = []
    m = len(pattern)
    g = string_to_int(pattern)
    h = string_to_int(string[:m])
    if g == h and pattern == string[:m]:
        matches.append(0)
    
    # pow(a,b,c) calcule efficacement a^b mod c
    #
    # il semble nécessaire pour la performance de mettre ce précalcul hors de la
    # boucle
    t = pow(4, m-1, mod)
    for i in range(len(string) - m):
        # Cela semble être légèrement plus rapide de faire la mise à jour de h
        # en une seule ligne, plutôt qu'avec une suite d'instructions.
        #
        # Aussi, on évite de faire appel à `string_to_int`: les appels aux
        # fonctions coûtent cher!
        h = ( 4*(h - pi[string[i]] * t) + pi[string[i+m]] ) % mod
        if g == h and pattern == string[i+1:i+m+1]:
            matches.append(i+1)
    return matches

Pas si mal, la perf!

In [6]:
%time m2 = rabin_karp(genome, 'GATTACA', 2**32 - 1)

CPU times: user 1.62 s, sys: 3.92 ms, total: 1.62 s
Wall time: 1.62 s


On vérifie que cela calcule la même sortie que l'algorithme naïf

In [7]:
m == m2

True

On va faire des tests avec d'autres motifs et paramètres, pour essayer de déterminer les meilleurs paramètres 

In [8]:
%time m2 = rabin_karp(genome, 'GATTACA', 2**16 - 1)

CPU times: user 1.74 s, sys: 0 ns, total: 1.74 s
Wall time: 1.74 s


In [9]:
aaaD = 'GTGAATATATCGAACAGTCAGGTTAACAGGCTGCGGCATTTTGTCCGCGCCGGGCTTCGCTCACTGTTCAGGCCGGAGCCACAGACCGCCGTTGAATGGGCGGATGCTAATTACTATCTCCCGAAAGAATCCGCATACCAGGAAGGGCGCTGGGAAACACTGCCCTTTCAGCGGGCCATCATGAATGCGATGGGCAGCGACTACATCCGTGAGGTGAATGTGGTGAAGTCTGCCCGTGTCGGTTATTCCAAAATGCTGCTGGGTGTTTATGCCTACTTTATAGAGCATAAGCAGCGCAACACCCTTATT'
aaaD

'GTGAATATATCGAACAGTCAGGTTAACAGGCTGCGGCATTTTGTCCGCGCCGGGCTTCGCTCACTGTTCAGGCCGGAGCCACAGACCGCCGTTGAATGGGCGGATGCTAATTACTATCTCCCGAAAGAATCCGCATACCAGGAAGGGCGCTGGGAAACACTGCCCTTTCAGCGGGCCATCATGAATGCGATGGGCAGCGACTACATCCGTGAGGTGAATGTGGTGAAGTCTGCCCGTGTCGGTTATTCCAAAATGCTGCTGGGTGTTTATGCCTACTTTATAGAGCATAAGCAGCGCAACACCCTTATT'

In [10]:
%time _ = match(genome, aaaD)

CPU times: user 2.62 s, sys: 0 ns, total: 2.62 s
Wall time: 2.62 s


In [11]:
%time _ = rabin_karp(genome, aaaD, 2**64 - 1)

CPU times: user 2.36 s, sys: 0 ns, total: 2.36 s
Wall time: 2.36 s


In [12]:
%time _ = rabin_karp(genome, aaaD, 2**32 - 1)

CPU times: user 2.28 s, sys: 0 ns, total: 2.28 s
Wall time: 2.28 s


In [13]:
%time _ = rabin_karp(genome, aaaD, 2**16 - 1)

CPU times: user 1.69 s, sys: 0 ns, total: 1.69 s
Wall time: 1.69 s


In [14]:
%time _ = rabin_karp(genome, aaaD, 2**8 - 1)

CPU times: user 1.61 s, sys: 3.97 ms, total: 1.61 s
Wall time: 1.61 s


In [15]:
accC = 'ATGCTGGATAAAATTGTTATTGCCAACCGCGGCGAGATTGCATTGCGTATTCTTCGTGCCTGTAAAGAACTGGGCATCAAGACTGTCGCTGTGCACTCCAGCGCGGATCGCGATCTAAAACACGTATTACTGGCAGATGAAACGGTCTGTATTGGCCCTGCTCCGTCAGTAAAAAGTTATCTGAACATCCCGGCAATCATCAGCGCCGCTGAAATCACCGGCGCAGTAGCAATCCATCCGGGTTACGGCTTCCTCTCCGAGAACGCCAACTTTGCCGAGCAGGTTGAACGCTCCGGCTTTATCTTCATTGGCCCGAAAGCAGAAACCATTCGCCTGATGGGCGACAAAGTATCCGCAATCGCGGCGATGAAAAAAGCGGGCGTCCCTTGCGTACCGGGTTCTGACGGCCCGCTGGGCGACGATATGGATAAAAACCGTGCCATTGCTAAACGCATTGGTTATCCGGTGATTATCAAAGCCTCCGGCGGCGGCGGCGGTCGCGGTATGCGCGTAGTGCGCGGCGACGCTGAACTGGCACAATCCATCTCCATGACCCGTGCGGAAGCGAAAGCTGCTTTCAGCAACGATATGGTTTACATGGAGAAATACCTGGAAAATCCTCGCCACGTCGAGATTCAGGTACTGGCTGACGGTCAGGGCAACGCTATCTATCTGGCGGAACGTGACTGCTCCATGCAACGCCGCCACCAGAAAGTGGTCGAAGAAGCGCCAGCACCGGGCATTACCCCGGAACTGCGTCGCTACATCGGCGAACGTTGCGCTAAAGCGTGTGTTGATATCGGCTATCGCGGTGCAGGTACTTTCGAGTTCCTGTTCGAAAACGGCGAGTTCTATTTCATCGAAATGAACACCCGTATTCAGGTAGAACACCCGGTTACAGAAATGATCACCGGCGTTGACCTGATCAAAGAACAGCTGCGTATCGCTGCCGGTCAACCGCTGTCGATCAAGCAAGAAGAAGTTCACGTTCGCGGCCATGCGGTGGAATGTCGTATCAACGCCGAAGATCCGAACACCTTCCTGCCAAGTCCGGGCAAAATCACCCGTTTCCACGCACCTGGCGGTTTTGGCGTACGTTGGGAGTCTCATATCTACGCGGGCTACACCGTACCGCCGTACTATGACTCAATGATCGGTAAGCTGATTTGCTACGGTGAAAACCGTGACGTGGCGATTGCCCGCATGAAGAATGCGCTGCAGGAGCTGATCATCGACGGTATCAAAACCAACGTTGATCTGCAGATCCGCATCATGAATGACGAGAACTTCCAGCATGGTGGCACTAACATCCACTATCTGGAGAAAAAACTCGGTCTTCAGGAAAAATAA'
accC

'ATGCTGGATAAAATTGTTATTGCCAACCGCGGCGAGATTGCATTGCGTATTCTTCGTGCCTGTAAAGAACTGGGCATCAAGACTGTCGCTGTGCACTCCAGCGCGGATCGCGATCTAAAACACGTATTACTGGCAGATGAAACGGTCTGTATTGGCCCTGCTCCGTCAGTAAAAAGTTATCTGAACATCCCGGCAATCATCAGCGCCGCTGAAATCACCGGCGCAGTAGCAATCCATCCGGGTTACGGCTTCCTCTCCGAGAACGCCAACTTTGCCGAGCAGGTTGAACGCTCCGGCTTTATCTTCATTGGCCCGAAAGCAGAAACCATTCGCCTGATGGGCGACAAAGTATCCGCAATCGCGGCGATGAAAAAAGCGGGCGTCCCTTGCGTACCGGGTTCTGACGGCCCGCTGGGCGACGATATGGATAAAAACCGTGCCATTGCTAAACGCATTGGTTATCCGGTGATTATCAAAGCCTCCGGCGGCGGCGGCGGTCGCGGTATGCGCGTAGTGCGCGGCGACGCTGAACTGGCACAATCCATCTCCATGACCCGTGCGGAAGCGAAAGCTGCTTTCAGCAACGATATGGTTTACATGGAGAAATACCTGGAAAATCCTCGCCACGTCGAGATTCAGGTACTGGCTGACGGTCAGGGCAACGCTATCTATCTGGCGGAACGTGACTGCTCCATGCAACGCCGCCACCAGAAAGTGGTCGAAGAAGCGCCAGCACCGGGCATTACCCCGGAACTGCGTCGCTACATCGGCGAACGTTGCGCTAAAGCGTGTGTTGATATCGGCTATCGCGGTGCAGGTACTTTCGAGTTCCTGTTCGAAAACGGCGAGTTCTATTTCATCGAAATGAACACCCGTATTCAGGTAGAACACCCGGTTACAGAAATGATCACCGGCGTTGACCTGATCAAAGAACAGCTGCGTATCGCTGCCGGTCAACCGCTGTCGATCAAGCAAGAAGAAGTTCACGTTCGCGGCCAT

In [16]:
%time _ = match(genome, accC)

CPU times: user 2.76 s, sys: 0 ns, total: 2.76 s
Wall time: 2.76 s


In [17]:
%time _ = rabin_karp(genome, accC, 2**64 - 1)

CPU times: user 2.25 s, sys: 0 ns, total: 2.25 s
Wall time: 2.25 s


In [18]:
%time _ = rabin_karp(genome, accC, 2**32 - 1)

CPU times: user 2.32 s, sys: 0 ns, total: 2.32 s
Wall time: 2.32 s


In [19]:
%time _ = rabin_karp(genome, accC, 2**16 - 1)

CPU times: user 1.88 s, sys: 0 ns, total: 1.88 s
Wall time: 1.88 s


In [20]:
%time _ = rabin_karp(genome, accC, 2**8 - 1)

CPU times: user 1.7 s, sys: 3.96 ms, total: 1.71 s
Wall time: 1.71 s
